In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn import metrics
from scipy.io import wavfile
import os
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import glob

Find Available GPUs

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


Load Data from Github

In [4]:
!git clone https://github.com/Andrew123098/MLDatabases.git

fatal: destination path 'MLDatabases' already exists and is not an empty directory.


Load Data from Files

In [4]:
# Load in 0 Degree .WAV Data.
X0 = []
count1 = 0
database_path = "MLDatabases/0"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X0.append(wavfile.read(filename)[1])
    count1 = count1 + 1

# Load in 60 Degree .WAV Data.
X60 = []
count1 = 0
database_path = "MLDatabases/60"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X60.append(wavfile.read(filename)[1])
    count1 = count1 + 1

# Load in 120 Degree .WAV Data.
X120 = []
count1 = 0
database_path = "MLDatabases/120"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X120.append(wavfile.read(filename)[1])
    count1 = count1 + 1

# Load in 180 Degree .WAV Data.
X180 = []
count1 = 0
database_path = "MLDatabases/180"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X180.append(wavfile.read(filename)[1])
    count1 = count1 + 1

# Load in 240 Degree .WAV Data.
X240 = []
count1 = 0
database_path = "MLDatabases/240"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X240.append(wavfile.read(filename)[1])
    count1 = count1 + 1

# Load in 300 Degree .WAV Data.
X300 = []
count1 = 0
database_path = "MLDatabases/300"
for filename in glob.glob(os.path.join(database_path, '*.wav')):
    X300.append(wavfile.read(filename)[1])
    count1 = count1 + 1

Resize and Reformat Data for Training

In [5]:
# Get the nth smallest size audio file (this will be sample size input to model)
n=5
X0 = sorted(X0,key=len)
X60 = sorted(X60,key=len)
X120 = sorted(X120,key=len)
X180 = sorted(X180,key=len)
X240 = sorted(X240,key=len)
X300 = sorted(X300,key=len)
X0=X0[n:]
X60=X60[n:]
X120=X120[n:]
X180=X180[n:]
X240=X240[n:]
X300=X300[n:]
sample_size = len(X0[0])
print(sample_size)
print("X0:",len(X0),"\nX300:",len(X300))

7168
X0: 154 
X300: 154


Split Audio Data into Equal Size Chunks and Concatenate into 1 Variable

In [6]:
# Make audio data into equal size chunks
X0e = []
for i in X0:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X0e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])
X0 = X0e
        
X60e = []
for i in X60:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X60e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])        
X60=X60e

X120e = []
for i in X120:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X120e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])        
X120=X120e

X180e = []
for i in X180:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X180e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])        
X180=X180e

X240e = []
for i in X240:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X240e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])        
X240=X240e

X300e = []
for i in X300:
    num_chunks = len(i)//sample_size
    for j in range(num_chunks):
        X300e.append(i[(j+1)*sample_size-sample_size:(j+1)*sample_size])        
X300=X300e

del X0e
del X60e
del X120e   
del X180e  
del X240e  
del X300e  

# Create Output data that is the same length as the input data.
Y0 = np.zeros([X0.__len__()],dtype='float32').tolist()
Y60 = (np.ones([X60.__len__()],dtype='float32')).tolist()
Y120 = (np.ones([X120.__len__()],dtype='float32')*2).tolist()
Y180 = (np.ones([X180.__len__()],dtype='float32')*3).tolist()
Y240 = (np.ones([X240.__len__()],dtype='float32')*4).tolist()
Y300 = (np.ones([X300.__len__()],dtype='float32')*5).tolist()

# Concatenate Left and Right .WAV data and output data as numpy arrays.
X0.extend(X60)
X0.extend(X120)
X0.extend(X180)
X0.extend(X240)
X0.extend(X300)
X = np.asarray(X0)
Y = np.asarray(Y0+Y60+Y120+Y180+Y240+Y300).astype(np.int16)
print("X",X.shape,"\nY:",Y.shape)

X (6096, 7168, 2) 
Y: (6096,)


Split Data into Test and Train Data of Sizes Divisible by Batch Size

In [7]:
# Make Divisible to Neaest 1000.
 
# Split data into test training data.          ## CHANGE TEST_SIZE HERE ##
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1666,random_state=0,shuffle=True,stratify=Y)

print("X_train:",X_train.shape,"\nY_train:",Y_train.shape,"\nX_test:",X_test.shape,"\nY_test:",Y_test.shape)

X_train: (5080, 7168, 2) 
Y_train: (5080,) 
X_test: (1016, 7168, 2) 
Y_test: (1016,)


In [8]:
# Split Data into Chunks of 1000
X_test = X_test[0:1000][:][:]
Y_test = Y_test[0:1000][:][:]
X_train = X_train[0:5000][:][:]
Y_train = Y_train[0:5000][:][:]

X_0 = X_test
Y_0 = Y_test
X_1 = X_train[0:1000][:][:]
Y_1 = Y_train[0:1000][:][:]
X_2 = X_train[1000:2000][:][:]
Y_2 = Y_train[1000:2000][:][:]
X_3 = X_train[2000:3000][:][:]
Y_3 = Y_train[2000:3000][:][:]
X_4 = X_train[3000:4000][:][:]
Y_4 = Y_train[3000:4000][:][:]
X_5 = X_train[4000:5000][:][:]
Y_5 = Y_train[4000:5000][:][:]

print("X_0",X_0.shape,"\nX_1",X_1.shape,"\nX_2:",X_2.shape,"\nX_3:",X_3.shape,"\nX_4:",X_4.shape,"\nX_5:",X_5.shape)

X_0 (1000, 7168, 2) 
X_1 (1000, 7168, 2) 
X_2: (1000, 7168, 2) 
X_3: (1000, 7168, 2) 
X_4: (1000, 7168, 2) 
X_5: (1000, 7168, 2)


5 Fold Cross Validation Option

In [9]:
# Set Fold from 0 to 5: 0 is no Cross Validation and 1-5 are cross validation folds.
fold = 0

if fold == 0:
    exit()

        
if fold == 1:
    X_test = X_1
    Y_test = Y_1
    X_train = np.concatenate((X_2,X_3,X_4,X_5),axis=0)
    Y_train = np.concatenate((Y_2,Y_3,Y_4,Y_5),axis=0)
    
if fold == 2:
    X_test = X_2
    Y_test = Y_2
    X_train = np.concatenate((X_1,X_3,X_4,X_5),axis=0)
    Y_train = np.concatenate((Y_1,Y_3,Y_4,Y_5),axis=0)
if fold == 3:
    X_test = X_3
    Y_test = Y_3
    X_train = np.concatenate((X_1,X_2,X_4,X_5),axis=0)
    Y_train = np.concatenate((Y_1,Y_2,Y_4,Y_5),axis=0)
if fold == 4:
    X_test = X_4
    Y_test = Y_4
    X_train = np.concatenate((X_1,X_2,X_3,X_5),axis=0)
    Y_train = np.concatenate((Y_1,Y_2,Y_3,Y_5),axis=0)
if fold == 5:
    X_test = X_5
    Y_test = Y_5
    X_train = np.concatenate((X_1,X_2,X_3,X_4),axis=0)
    Y_train = np.concatenate((Y_1,Y_2,Y_3,Y_4),axis=0)        

# Number of Output Categories
n_classes = 6

# Print Train and Test Split Shapes
print("X_train:",X_train.shape,"\nY_train:",Y_train.shape,"\nX_test:",X_test.shape,"\nY_test:",Y_test.shape)

X_train: (5000, 7168, 2) 
Y_train: (5000,) 
X_test: (1000, 7168, 2) 
Y_test: (1000,)


Create and Compile the Model

In [ ]:
# Create the Model
model = Sequential()

# Add a LSTM layer with 1 output, and ambiguous input data length.
model.add(layers.LSTM(256,input_shape=(sample_size,2),return_sequences=True,kernel_regularizer='l2'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5)) #dropout with 50% rate
model.add(layers.LSTM(128,return_sequences=True,kernel_regularizer='l2'))
model.add(layers.BatchNormalization())
model.add(layers.Bidirectional(layers.GRU(64,return_sequences=True,kernel_regularizer='l2')))
model.add(layers.BatchNormalization())
model.add(layers.Bidirectional(layers.GRU(32,return_sequences=False,kernel_regularizer='l2')))
model.add(layers.BatchNormalization())
model.add(layers.Dense(16,activation='relu',kernel_regularizer='l2'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(n_classes, activation='softmax',kernel_regularizer='l2'))

# Compile Model
optimizer = Adam(learning_rate=2*1e-5)
history = model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"]
)
model.summary()

Define Training Parameters and Run

In [ ]:
# Define Training Parameters
num_epochs = 8
num_batch_size = 200

# Save the most accurate model to file. (Verbosity Gives more information)
checkpointer = ModelCheckpoint(filepath="SavedModels/checkpointModel.hdf5", verbose=1,save_best_only=True)

# Start the timer
start = datetime.now()

# Train the model
model.fit(X_train,Y_train,batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test,Y_test), callbacks=[checkpointer],verbose=1)

# Get and Print Model Validation Accuracy
test_accuracy=model.evaluate(X_test,Y_test,verbose=0)
print(test_accuracy[1])

Convert Saved Model to C Code using tflite.

In [ ]:
loaded_model = tf.keras.models.load_model('SavedModels/checkpointModel.hdf5')
converter = lite.TFLiteConverter.from_keras_model(loaded_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('new_model.tflite', 'wb').write(tfmodel)